# Basic Car Data Wrangling   

Start by importing certain libraries:

In [126]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

#Also ignoring the warnings for neatness purposes:
import warnings
warnings.filterwarnings('ignore')

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [127]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

**Importing the dataset from downloads:** <br>

In [128]:
cars = pd.read_csv('Sport car price.csv')

View the first 2 rows just to see what we are dealing with:

In [129]:
cars.iloc[:2]

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
0,Porsche,911,2022,3,379,331,4,"101,200"
1,Lamborghini,Huracan,2021,5.2,630,443,2.8,"274,390"


In [130]:
print("Length of Uncleaned Data:", len(cars))

Length of Uncleaned Data: 1007


**Dealing with NaN values:** <br> 
- First, I am going to see how many rows contain NaN values:

In [131]:
checking_NaN = cars.dropna()
print("Length of Data without NaN values:", len(checking_NaN))

Length of Data without NaN values: 995


- Since only 12 rows are with missing values, I am going to just drop these instead of fill in the values:

In [132]:
cars = cars.dropna()

### **Summary Statistics:** 
To start I have just used a summary statistics table with the IQR, max, min, average, etc. Right now this is not too useful, but does allow us to gain a better picture about the data. 

NB: If I .describe() the data straight away, I only get 'Year' because the rest have been inputted as strings, so I am going to float the others first. 

In [133]:
cars['Horsepower'] = pd.to_numeric(cars['Horsepower'].replace(',', ''), errors='coerce')
cars['Torque (lb-ft)'] = pd.to_numeric(cars['Torque (lb-ft)'].replace('-', ''), errors='coerce')
cars['0-60 MPH Time (seconds)'] = pd.to_numeric(cars['0-60 MPH Time (seconds)'].replace('-', ''), errors='coerce')
cars['Price (in USD)'] = pd.to_numeric(cars['Price (in USD)'].str.replace(',', ''), errors='coerce')
cars['Engine Size (L)'] = pd.to_numeric(cars['Engine Size (L)'], errors='coerce')
cars.loc[cars['Engine Size (L)'].isnull(), 'Engine Size (L)'] = 'Electric Motor'

cars[:2]

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
0,Porsche,911,2022,3.00,379.00,331.00,4.00,101200
1,Lamborghini,Huracan,2021,5.20,630.00,443.00,2.80,274390


In [134]:
#I have used the .T to switch the column and row titles so it helps with our readability:
cars.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,995.00,2021.20,2.03,1965.00,2021.00,2021.00,2022.00,2023.00
Horsepower,988.00,621.01,289.04,181.00,454.00,591.00,671.00,2000.00
Torque (lb-ft),990.00,537.32,232.15,151.00,406.00,507.00,597.50,1732.00
0-60 MPH Time (seconds),994.00,3.53,0.77,1.80,2.90,3.50,4.00,6.50
Price (in USD),995.00,375420.02,730441.94,25000.00,71800.00,135000.00,249000.00,5200000.00


**Takeaways From the Table:** 
- The mean year for cars is 2021, but the minimum is 1965. With the 75% percentile as 2022, the mean is heavily skewed to the present day. 
- The range for 0-60MPH is large, (6.5-1.8)
- The price range is bigger, with the minimum car being worth 25,000USD, versus 5,200,000USD!

**Starting with basic groupby functions and plotting counts:**
- I have chosen to use the groupby function and basic boxplots to take a better look at the columns with strings.

In [105]:
grouped_by_make = cars.groupby(['Car Make'])[['Horsepower', 'Torque (lb-ft)', '0-60 MPH Time (seconds)', 'Price (in USD)']].agg(['mean'])
pd.options.display.float_format = '{:.2f}'.format

grouped_by_make = grouped_by_make.sort_values(by=('Price (in USD)', 'mean'), ascending=False)

The four most expensive car makes (by mean price) are listed below:

In [106]:
grouped_by_make[:4]

,Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
,mean,mean,mean,mean
Car Make,,,,
Bugatti,1502.32,1180.00,2.39,3251956.52
Koenigsegg,1364.13,1069.60,2.47,2906666.67
Pagani,758.33,744.00,2.80,2791666.67
Pininfarina,1873.00,1696.00,1.90,2500000.00


The four cheapest car makes (by mean price) are listed below:

In [107]:
grouped_by_make[-4:]

,Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
,mean,mean,mean,mean
Car Make,,,,
Nissan,360.49,287.62,4.75,50752.16
Toyota,382.00,368.00,3.90,43072.00
Subaru,310.00,290.00,5.00,38170.00
Mazda,181.00,151.00,6.50,26830.00


**Comparing the mean figures of the top four and bottom four cars by price:**
- Horsepower is considerably higher for the more expensive cars, though Pagani lags behind
- While Toyota is the fastest, cheapest car, it is still over 1 second slower than the slowest, expensive car
- The mean price of the most expensive and cheapest cars range by almost $3,000,000.

**Looking at the most frequent car makes:**

In [108]:
cars['Car Make'].value_counts()[:10]

Porsche          86
McLaren          75
Audi             71
Lamborghini      66
BMW              63
Chevrolet        60
Ferrari          55
Mercedes-Benz    54
Aston Martin     50
Ford             48
Name: Car Make, dtype: int64

In [109]:
cars['Car Make'].value_counts()[-10:]

Subaru         3
Pininfarina    2
TVR            2
Kia            1
Alpine         1
Ariel          1
Shelby         1
Mazda          1
Polestar       1
Ultima         1
Name: Car Make, dtype: int64

- Interestingly, Porsche is the most represented car make in this dataset. Mclaren is next. This is interesting to me because they are both considered luxury cars. I would have expected our cheapted four cars to be at the top of this list.
- Instead, Subaru, one of the four cheapest cars is one of the least purchased, as is Mazda.

**Sorting by Horsepower instead of Speed:**

In [110]:
grouped_by_speed = cars.groupby(['Car Make'])[['Horsepower', 'Torque (lb-ft)', '0-60 MPH Time (seconds)', 'Price (in USD)']].agg(['mean'])
pd.options.display.float_format = '{:.2f}'.format

grouped_by_speed = grouped_by_speed.sort_values(by=('Horsepower', 'mean'), ascending=False)

In [111]:
grouped_by_speed[:4]

,Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
,mean,mean,mean,mean
Car Make,,,,
Rimac,1911.64,1702.55,1.87,2400000.00
Pininfarina,1873.00,1696.00,1.90,2500000.00
Bugatti,1502.32,1180.00,2.39,3251956.52
Koenigsegg,1364.13,1069.60,2.47,2906666.67


In [112]:
grouped_by_speed[-4:]

,Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
,mean,mean,mean,mean
Car Make,,,,
Ariel,320.00,243.00,2.80,75000.00
Subaru,310.00,290.00,5.00,38170.00
Alpine,288.00,236.00,4.40,71500.00
Mazda,181.00,151.00,6.50,26830.00


- Pininfarina, Bugatti, Koenigsegg are three of the fastest cars, and are in the top four expensive cars.
- Conversely, Mazda and Subaru are two of the cheapest cars, and slowest.

In [116]:
grouped_by_make_year = cars.groupby(['Car Make', 'Car Model', 'Year'])[['Horsepower', 'Torque (lb-ft)', '0-60 MPH Time (seconds)', 'Price (in USD)']].agg(['mean'])
grouped_by_make_year[10:19]

Horsepower Torque (lb-ft)  \
                                         mean           mean   
Car Make     Car Model        Year                             
Aston Martin DBS Superleggera 2021     715.00         663.64   
                              2022     715.00         663.50   
             Vantage          2021     503.00         505.00   
                              2022     503.00         505.00   
Audi         R8               2022     562.00         406.00   
             R8 Spyder        2022     562.00         406.00   
             RS 3             2022     401.00         361.50   
             RS 5             2021     444.00         443.00   
                              2022     444.00         442.75   

                                   0-60 MPH Time (seconds) Price (in USD)  
                                                      mean           mean  
Car Make     Car Model        Year                                         
Aston Martin DBS Superleggera 2021                    3.27      306494.09  
                              2022                    3.30      318403.33  
             Vantage          2021                    3.54      145772.38  
                              2022                    3.52      146527.00  
Audi         R8               2022                    3.20      142700.00  
             R8 Spyder        2022                    3.50      183300.00  
             RS 3             2022                    3.55       56100.00  
             RS 5             2021                    3.50       75100.00  
                              2022                    3.55       75575.00

In [124]:
grouped_by_make_year[52:54]

Horsepower Torque (lb-ft) 0-60 MPH Time (seconds)  \
                              mean           mean                    mean   
Car Make Car Model Year                                                     
BMW      M5        2021     600.00         553.00                    3.20   
                   2022     600.00         553.00                    3.20   

                        Price (in USD)  
                                  mean  
Car Make Car Model Year                 
BMW      M5        2021      103500.00  
                   2022      103571.43

**Comparing Cars over time:**
- Above, I have grouped by car make and model. 
- The Aston Martin DBS Superleggera 2021 and 2022 versions are practically the same in terms of speed, horsepower, and torque. However, the price is slightly lower for the 2021 model.
- The Audi RS5 has the same result as the Aston Martin, with the cars being the same, but cheaper for 2021.
- The BMW M5 is **exactly** the same, but the 2021 version is slightly cheaper.

These comparisons lead me to believe that the older the car, the cheaper it is, even if it has the same qualities as the newer-year cars.